# Hourly dataset for the project 


### NORD MACROZONE

In [96]:
import sys
print(sys.executable)

c:\ProgramData\anaconda3\python.exe


In [97]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_nord = qh[qh['MACROZONA'] == 'NORD']


qh_nord.tail()

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,MACROZONA,SBIL_MWH
ORAINI,,
2024-12-04 14:00:00,NORD,-110.423
2024-12-04 14:15:00,NORD,-147.133
2024-12-04 14:30:00,NORD,-185.204
2024-12-04 14:45:00,NORD,-136.905
2024-12-04 15:00:00,NORD,207.420


In [98]:
# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour
h_nord = qh_nord.resample('H').sum()

# Clean the 'MACROZONA' column to retain only 'NORD'
h_nord['MACROZONA'] = h_nord['MACROZONA'].str[:4]

h_nord = h_nord.drop(columns = ['MACROZONA'])
# Display the first few rows of the resampled data
h_nord.tail()

C:\Users\samueledelia\AppData\Local\Temp\3\ipykernel_10056\4018051853.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  h_nord = qh_nord.resample('H').sum()


,SBIL_MWH
ORAINI,
2024-12-04 11:00:00,657.699
2024-12-04 12:00:00,1549.423
2024-12-04 13:00:00,-313.293
2024-12-04 14:00:00,-579.665
2024-12-04 15:00:00,207.420


In [99]:
lags = [-1, -2, -3, -24]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(h_nord['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged = df_sbil_lagged.resample('H').sum()

df_sbil_lagged

C:\Users\samueledelia\AppData\Local\Temp\3\ipykernel_10056\1260965639.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sbil_lagged = df_sbil_lagged.resample('H').sum()


,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24
ORAINI,,,,
2019-01-01 00:00:00,0.000,0.000,0.000,0.000
2019-01-01 01:00:00,-111.900,0.000,0.000,0.000
2019-01-01 02:00:00,-112.451,-111.900,0.000,0.000
2019-01-01 03:00:00,-88.849,-112.451,-111.900,0.000
2019-01-01 04:00:00,127.792,-88.849,-112.451,0.000
...,...,...,...,...
2024-12-04 11:00:00,-61.465,-582.215,-175.515,291.362
2024-12-04 12:00:00,657.699,-61.465,-582.215,-6.472
2024-12-04 13:00:00,1549.423,657.699,-61.465,-300.884


In [100]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,1843.786,0.0,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,1845.050,0.0,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,1845.647,0.0,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,983.111,0.0,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,978.775,0.0,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-05 19:00:00,42458.793,43532.177,3660.670,7724.038,23352.455,1128.925,2380.449,2541.671,0.0,None,...,NaN,0.0,3934.000,0.0,0.0,240.0,3385.0,39907.177,43118.793,2024-12-04 13:10:37.180
2024-12-05 20:00:00,40433.658,41465.458,3384.412,7455.731,22053.846,1123.659,2351.426,2469.320,0.0,None,...,NaN,0.0,3912.000,0.0,0.0,240.0,3385.0,37840.458,41093.658,2024-12-04 13:10:37.180
2024-12-05 21:00:00,37419.551,37890.203,3096.806,6825.088,20501.039,1048.671,2187.339,2259.712,0.0,None,...,NaN,0.0,3709.780,572.7,0.0,239.0,3375.0,34276.190,37419.551,2024-12-04 13:10:37.180


In [101]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mgp_volumes_nord.rename(columns={"NORD_PURCHASES": "MGP_NORD_PURCHASES", "NORD_SALES": "MGP_NORD_SALES"}, inplace=True)
mgp_volumes_nord = mgp_volumes_nord[~mgp_volumes_nord.index.duplicated(keep='first')]


#mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
#mi1_volumes = mi1_volumes.drop(columns=['MARKET'])
#mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
#mi1_volumes_nord.rename(columns={"NORD_PURCHASES": "MI1_NORD_PURCHASES", "NORD_SALES": "MI1_NORD_SALES"}, inplace=True)
#mi1_volumes_nord = mi1_volumes_nord[~mi1_volumes_nord.index.duplicated(keep='first')]

In [102]:
#mi1_volumes_nord.tail(40)

In [103]:
# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16_
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = """
SELECT * FROM POWER_UNBALANCE
"""
power_curve = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power_curve = power_curve.sort_values(by='TIMESTAMP')

power_curve = power_curve.pivot(index='TIMESTAMP', columns='SOURCE_ZONE', values='UNBALANCE_kW')

# Rename columns for clarity (optional, if needed)
power_curve.columns = [f"UNBALANCE_{col}" for col in power_curve.columns]

# Display the resulting DataFrame
power_curve = power_curve.resample('H').sum()

power_curve = power_curve[["UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD", "UNBALANCE_IDRO-PROGRAMMABILE_NORD", "UNBALANCE_SOLARE_NORD"]]

power_curve = power_curve.rename_axis("ORAINI")

power_curve

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\3\ipykernel_10056\3830827227.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  power_curve = power_curve.resample('H').sum()


,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,
2023-11-01 01:00:00,0.00,12447.43,0.000
2023-11-01 02:00:00,0.00,12130.31,0.000
2023-11-01 03:00:00,0.00,12297.07,0.000
2023-11-01 04:00:00,0.00,25895.64,0.000
2023-11-01 05:00:00,0.00,-28886.10,0.000
...,...,...,...
2024-12-04 11:00:00,114684.66,112148.49,179698.422
2024-12-04 12:00:00,-18662.62,6695.82,43614.430
2024-12-04 13:00:00,1392.66,6034.13,-78985.860


In [104]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM ENTSOE_DATA" 
entsoe_data = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
entsoe_data = entsoe_data.sort_values(by='ORAINI')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [105]:
entsoe_data = entsoe_data[entsoe_data['DOMAIN'] == 'LOAD']
entsoe_data = entsoe_data.drop(columns=['DOMAIN'])
# Convert ORAINI to datetime using the correct method
entsoe_data['ORAINI'] = pd.to_datetime(entsoe_data['ORAINI'], format='%Y%m%d%H%M')
entsoe_data.set_index('ORAINI', inplace=True)

day_ahead_load = entsoe_data[entsoe_data['SCOPE'] == 'DAY AHEAD']
day_ahead_load = day_ahead_load[['CODZONA', 'VALUE']]
day_ahead_load["FORECAST_TOTAL_LOAD_MW"] = day_ahead_load["VALUE"]
day_ahead_load = day_ahead_load.drop(columns="VALUE")

actual_load = entsoe_data[entsoe_data['SCOPE'] == 'ACTUAL']
actual_load = actual_load[['CODZONA', 'VALUE']]
actual_load["TOTAL_LOAD_MW"] = actual_load["VALUE"]
actual_load = actual_load.drop(columns="VALUE")

entsoe_load = pd.merge(day_ahead_load, actual_load, on=['ORAINI', 'CODZONA'])

subset_zonas = ['NORD']
mnord_load = entsoe_load[entsoe_load['CODZONA'].isin(subset_zonas)]
mnord_load = mnord_load.drop(columns=['CODZONA'])
mnord_load


,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,11110.0,11511.0
2019-01-01 01:00:00,10675.0,11039.0
2019-01-01 02:00:00,10080.0,10430.0
2019-01-01 03:00:00,9793.0,9811.0
2019-01-01 04:00:00,9485.0,9474.0
...,...,...
2024-12-04 09:00:00,26441.0,27113.0
2024-12-04 10:00:00,25829.0,26456.0
2024-12-04 11:00:00,25375.0,25631.0


In [106]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [h_nord, df_sbil_lagged, mgp_volumes_nord, mnord_load, power_curve] #mi1_volumes_nord
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord_h_project = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

#df_nord_h = df_nord.drop(columns="MACROZONA")
df_nord_h_project = df_nord_h_project[df_nord_h_project.index >= '2024-08-27']
df_nord_h_project

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,MGP_NORD_PURCHASES,MGP_NORD_SALES,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,,,,,,,,,,
2024-08-27 00:00:00,-6.856,-97.279,-51.994,-409.274,-270.368,16126.496,12677.375,17358.0,16029.0,-282467.38,0.0,0.0
2024-08-27 01:00:00,35.347,-6.856,-97.279,-51.994,-147.570,15370.564,11892.387,16570.0,15261.0,-287513.36,0.0,0.0
2024-08-27 02:00:00,-32.585,35.347,-6.856,-97.279,-214.758,14943.335,11202.301,16123.0,14842.0,-352965.19,0.0,0.0
2024-08-27 03:00:00,-117.998,-32.585,35.347,-6.856,21.760,14713.801,10895.785,15720.0,14628.0,-240815.93,0.0,0.0
2024-08-27 04:00:00,-297.574,-117.998,-32.585,35.347,-157.722,14760.303,11026.509,15615.0,14826.0,-255695.64,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-05 19:00:00,NaN,NaN,NaN,NaN,NaN,23352.455,18962.100,NaN,NaN,NaN,NaN,NaN
2024-12-05 20:00:00,NaN,NaN,NaN,NaN,NaN,22053.846,17317.978,NaN,NaN,NaN,NaN,NaN
2024-12-05 21:00:00,NaN,NaN,NaN,NaN,NaN,20501.039,14817.090,NaN,NaN,NaN,NaN,NaN


In [107]:
df_nord_h_project.tail(48)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,MGP_NORD_PURCHASES,MGP_NORD_SALES,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,,,,,,,,,,
2024-12-04 00:00:00,75.801,-564.841,-694.867,-560.133,-95.727,15561.030,11192.735,15726.0,15514.0,-27098.97,68565.17,2349.600
2024-12-04 01:00:00,91.645,75.801,-564.841,-694.867,50.557,15273.771,10506.773,14921.0,14735.0,-10569.68,5619.25,2349.600
2024-12-04 02:00:00,32.192,91.645,75.801,-564.841,383.856,14863.279,10422.818,14489.0,14347.0,23132.99,7236.15,2349.600
2024-12-04 03:00:00,-88.035,32.192,91.645,75.801,15.486,14958.721,10034.710,14510.0,14385.0,-5036.10,6885.23,2349.600
2024-12-04 04:00:00,-162.976,-88.035,32.192,91.645,-161.689,14915.849,9915.423,14800.0,14692.0,-11274.66,9770.17,2349.600
2024-12-04 05:00:00,300.574,-162.976,-88.035,32.192,-52.337,15844.043,11306.607,16065.0,16148.0,-69861.25,8688.67,2349.600
2024-12-04 06:00:00,446.800,300.574,-162.976,-88.035,102.807,19393.910,16682.004,19573.0,19724.0,-65390.75,6911.37,2349.600
2024-12-04 07:00:00,14.629,446.800,300.574,-162.976,-642.885,22801.171,20605.974,23721.0,24175.0,22912.62,7014.87,-963.126
2024-12-04 08:00:00,-175.515,14.629,446.800,300.574,-631.357,25158.116,23222.123,26143.0,26666.0,39689.36,-49145.67,-15047.357


In [108]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord_h_project.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord_h_project.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\3\ipykernel_10056\2037195458.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord_h_project.csv')


### SUD MACROZONE

In [109]:
"""
import sys
print(sys.executable)"""

'\nimport sys\nprint(sys.executable)'

In [110]:
"""import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_sud = qh[qh['MACROZONA'] == 'SUD']


qh_sud.tail()"""

'import os\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# Add the source directory to the system path\nsys.path.append(os.path.abspath(\'../src\'))\nfrom open_data import fetch_db_table_sqlserver16\nimport utils\nimport importlib\n\n# Reload the module\nimportlib.reload(utils)\n\n# SQL query to fetch data\nsql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" \nqh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)\nqh = qh.sort_values(by=\'ORAINI\')\n# Filter the DataFrame for the relevant columns and rows\nqh = qh[[\'MACROZONA\', \'ORAINI\', \'SBIL_MWH\']]\n# Convert ORAINI to datetime using the correct method\nqh[\'ORAINI\'] = pd.to_datetime(qh[\'ORAINI\'], format=\'%Y%m%d%H%M\')\nqh.set_index(\'ORAINI\', inplace=True)\n# Nord\nqh_sud = qh[qh[\'MACROZONA\'] == \'SUD\']\n\n\nqh_sud.tail()'

In [111]:
"""# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour
h_sud = qh_sud.resample('H').sum()

# Clean the 'MACROZONA' column to retain only 'NORD'
h_sud['MACROZONA'] = h_sud['MACROZONA'].str[:4]

h_nord = h_nord.drop(columns = ['MACROZONA'])
# Display the first few rows of the resampled data
h_nord.tail()"""

"# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour\nh_sud = qh_sud.resample('H').sum()\n\n# Clean the 'MACROZONA' column to retain only 'NORD'\nh_sud['MACROZONA'] = h_sud['MACROZONA'].str[:4]\n\nh_nord = h_nord.drop(columns = ['MACROZONA'])\n# Display the first few rows of the resampled data\nh_nord.tail()"